<a href="https://colab.research.google.com/github/bk-anupam/KaggleChallenges/blob/master/NLP/PatentPhraseMatching/USPPPM_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu May  5 05:56:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%pip install -q transformers[sentencepiece] datasets

     |████████████████████████████████| 4.0 MB 8.2 MB/s 
     |████████████████████████████████| 325 kB 72.5 MB/s 
     |████████████████████████████████| 136 kB 72.0 MB/s 
     |████████████████████████████████| 212 kB 76.2 MB/s 
     |████████████████████████████████| 77 kB 7.1 MB/s 
     |████████████████████████████████| 1.1 MB 55.7 MB/s 
     |████████████████████████████████| 127 kB 74.0 MB/s 
     |████████████████████████████████| 271 kB 70.9 MB/s 
     |████████████████████████████████| 144 kB 67.2 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 880 kB 68.8 MB/s 
     |████████████████████████████████| 6.6 MB 53.3 MB/s 
     |████████████████████████████████| 596 kB 59.2 MB/s 
     |████████████████████████████████| 1.2 MB 59.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6

In [4]:
import numpy as np
import pandas as pd
import torch
import os
from sklearn.model_selection import StratifiedKFold
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from functools import partial
import warnings

warnings.filterwarnings('ignore')


In [5]:
class TrainingArgs:
    weight_decay = 0.01
    learning_rate = 2e-5  
    warmup_ratio = 0.1
    gradient_accumulation_steps = 8
    fp16 = True
    lr_scheduler_type = "cosine"

class Config:
    DATA_PATH = "/content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/data/"
    RANDOM_STATE = 42
    BATCH_SIZE = 16
    NUM_LABELS = 1
    NUM_FOLDS = 1
    NUM_EPOCHS = 4
    NUM_WORKERS = 8
    TRANSFORMER_CHECKPOINT = "microsoft/deberta-v3-large"
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    OUT_DIR = "/content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/model/"

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [6]:
df_train = pd.read_csv(Config.DATA_PATH + "train.csv")
df_test = pd.read_csv(Config.DATA_PATH + "test.csv")
df_titles = pd.read_csv(Config.DATA_PATH + "titles.csv")

In [7]:
df_test = pd.merge(
    left = df_test,
    right = df_titles[["code", "title"]],
    how = "inner",
    left_on = "context",
    right_on = "code"
)

In [8]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained(Config.TRANSFORMER_CHECKPOINT)
# DataCollatorWithPadding pads each batch to the longest sequence length
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/580 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
sep = tokenizer.sep_token
df_test["inputs"] = df_test.anchor + sep + df_test.target + sep + df_test.title 
df_test.head()

,id,anchor,target,context,code,title,inputs
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,G02,OPTICS,opc drum[SEP]inorganic photoconductor drum[SEP...
1,5203a36c501f1b7c,generate in layer,generate by layer,G02,G02,OPTICS,generate in layer[SEP]generate by layer[SEP]OP...
2,7aa5908a77a7ec24,el display,illumination,G02,G02,OPTICS,el display[SEP]illumination[SEP]OPTICS
3,09e418c93a776564,adjust gas flow,altering gas flow,F23,F23,COMBUSTION APPARATUS; COMBUSTION PROCESSES,adjust gas flow[SEP]altering gas flow[SEP]COMB...
4,36baf228038e314b,lower trunnion,lower locating,B60,B60,VEHICLES IN GENERAL,lower trunnion[SEP]lower locating[SEP]VEHICLES...


In [10]:
def tokenize_text(tokenizer, with_labels, row):
    encoding = tokenizer(
        text = row["inputs"],
        padding = False,
        truncation = True
    )
    if with_labels:
        encoding["labels"] = row["score"]
    return encoding

In [11]:
preprocess_test_data = partial(tokenize_text, tokenizer, False)  
ds_test_raw = Dataset.from_pandas(df_test)
raw_ds_col_names = ds_test_raw.column_names  
ds_test = ds_test_raw.map(preprocess_test_data, batched=True, batch_size=1000, remove_columns=raw_ds_col_names)  

  0%|          | 0/1 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [12]:
dl_test = DataLoader(ds_test, batch_size=Config.BATCH_SIZE, collate_fn=data_collator, num_workers=Config.NUM_WORKERS)

In [13]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.reshape(len(predictions))
    return {
        'pearson': np.corrcoef(predictions, labels)[0][1]
    }

In [14]:
from transformers import DebertaV2ForSequenceClassification

def get_fold_model():
    model = DebertaV2ForSequenceClassification.from_pretrained(Config.OUT_DIR + "checkpoint-912", local_files_only=True)
    model.to(Config.DEVICE)
    return model

In [15]:
import gc

for fold in range(Config.NUM_FOLDS):
    model = get_fold_model()
    trainer = Trainer(model=model, tokenizer=tokenizer)
    outputs = trainer.predict(ds_test)
    del model, trainer
    gc.collect()
    torch.cuda.empty_cache()
    break

***** Running Prediction *****
  Num examples = 36
  Batch size = 8


loading configuration file /content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/model/checkpoint-912/config.json
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-large",
  "architectures": [
    "DebertaV2ForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 1024,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "torc

loading configuration file /content/gdrive/MyDrive/Kaggle/NLP/PatentPhraseMatching/model/checkpoint-912/config.json
Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-large",
  "architectures": [
    "DebertaV2ForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 1024,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "torc

KeyboardInterrupt: ignored

In [16]:
# from tqdm.auto import tqdm

# # perform predictions on test data
# test_preds = []
# with torch.no_grad():
#     for step, batch in tqdm(enumerate(dl_test)):
#         batch = {k: v.to(Config.DEVICE) for k, v in batch.items()}           
#         outputs = model(**batch)
#         predictions = outputs.logits.argmax(dim=-1)         
#         test_preds.extend(predictions.cpu().detach().numpy().tolist())
#         break

# print(f"Completed prediction for {len(test_preds)} test rows")

In [26]:
df_submission = pd.DataFrame({"id": df_test.id, "score": outputs.predictions.reshape(-1)})
df_submission

,id,score
0,4112d61851461f60,0.583688
1,5203a36c501f1b7c,0.941375
2,7aa5908a77a7ec24,0.309159
3,09e418c93a776564,0.742090
4,36baf228038e314b,0.585920
5,b892011ab2e2cabc,0.638935
6,1f37ead645e7f0c8,0.347646
7,71a5b6ad068d531f,0.013066
8,16ae4b99d3601e60,0.242842
9,474c874d0c07bd21,0.515820


In [27]:
df_submission.to_csv(Config.DATA_PATH + "submission.csv", index=False)